# Advanced Topics in Urban Informatics || Greg Dobler || June 13, 2016
-----------------------------------------------------------

## Structure of this class
Basics of image processing and computer vision with useful tool building along the way <br>
One homework problem which will consist of submitting code <br>
Useful reading: <i>Programming Computer Vision with Python</i>, Jan Erik Solem

<br>

## Getting started with scientific python
NumPy -- arrays (2D and 3D), vectorized operations, basic functionality <br>
SciPy -- functions, transformations, analysis <br>
Matplotlib -- plotting, viewing, visualizations (static and interactive)

<br>

## Image processing and Computer Vision
What are images from a data perspective? <br>
How are images loaded displayed and handled? <br>
What are some basic image processing taks? <br>
What can we learn from images? <br><br>
<b>Video is not particularly <i>special</i>, it's just a series of images to be analyzed together</b>

----------------------

## 1D
Let's look at a "time series".  First some imports that will be useful:

In [2]:
import os
import sys
import time
import numpy as np
import scipy.ndimage as nd
import matplotlib.pyplot as plt

# -- helps for interactive plotting with notebooks
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


Now we load in the one-dimensional data (which has been saved as a numpy array) and define a plotting function to view the data in a couple of different ways.

In [3]:
data = np.load('output/ml_flat.npy')

In [4]:
fig0 = plt.figure(0)
def showit(fac,sep=100.):
    fig0.clf()
    ax0 = fig0.add_subplot(1,1,1)
    lin0 = ax0.plot(range(data.size/fac), 
                    data.reshape(data.size/fac,fac) + 
                    sep*np.arange(fac),lw=0.05,color='k')
    fig0.canvas.draw()

In [5]:
showit(1)

This one-dimensional "time series" can be sliced and plotted stacked on top of itself.  For example, cut it in half and plot the two curves:

In [6]:
showit(2,200)

If we progressively cut further, stacking each time..

In [7]:
showit(954,10)

In [10]:
fig1, ax1 = plt.subplots(num=1)
im1 = ax1.imshow(data.reshape(data.size/954,954).T[::-1,:])
ax1.grid(0)
fig1.canvas.draw()

For *many* reasons, displaying 2D information in a rainbow color scheme is a poor choice.  Here are some good defaults to set:

In [9]:
plt.rcParams["image.cmap"] = "gist_gray"
plt.rcParams["image.interpolation"] = "nearest"

---

# 1D vs 2D arrays

Grayscale images are 2D <b>arrays</b> but in many cases can be thought of (and treated as) 1D arrays.<br>
<br>
Many data analysis tasks you've already seen can be handled exactly the same:<br>
. correlations<br>
. means and variances<br>
. multivariate regressions<br>
<br>
others will be different:<br>
. filters<br>
. derivatives<br>
. rotations<br>
<br>
But since this is a numpy array things are easily <b><i>vectorized</i></b> and fast.  For example, let's look at derivatives.  First the 1D case:

In [11]:
xx = np.linspace(-20,20,1000)
yy = np.cos(xx)

In [12]:
fig3, ax3 = plt.subplots(num=3)
lin3a, = ax3.plot(xx,yy,'darkred')
lin3b, = ax3.plot(xx[1:],(yy[1:]-yy[:-1])/(xx[1]-xx[0]),
                  'dodgerblue')
fig3.canvas.draw()

Which can be used to find edges,

In [13]:
xx = np.linspace(-20,20,1000)
yy = 1.0*(xx>4.3)

In [14]:
lin3a.set_data(xx,yy)
lin3b.set_data(xx[1:],(yy[1:]-yy[:-1])/(xx[1]-xx[0]))
lin3b.set_visible(False)
ax3.set_ylim(-5,5)
fig3.canvas.draw()

In [15]:
lin3b.set_visible(True)
fig3.canvas.draw()

The same thing is true in 2D, but now we can do all rows at once,

In [16]:
img = data.reshape(data.size/954,954).T[::-1,:]

In [18]:
fig4, ax4 = plt.subplots(1,2,num=4)
im4a = ax4[0].imshow(img)
im4b = ax4[1].imshow(img[:,5:]-img[:,:-5])
fig4.canvas.draw()

... or we can do all columns at once

In [19]:
im4b.set_data(img[5:,:]-img[:-5,:])
fig4.canvas.draw()

For completeness, let's do rows and columns (not simultaneously!) take the absolute value and add them together.  This is the simplest version of an edge detector,

In [20]:
edges_simp = np.abs(img[5:,5:]-img[:-5,5:]) + \
    np.abs(img[5:,5:]-img[5:,:-5])

In [21]:
im4b.set_data(edges_simp)
fig4.canvas.draw()

which is often combined with thresholding

In [22]:
im4b.set_data(edges_simp>60)
im4b.set_clim(0,1)
fig4.canvas.draw()

You can see it's not doing great, it's missing some edges and there's a lot of noise.  We'll fix this later...

---

(always be aware of your imports and <b><u><i>preserve namespaces</i></u></b>!!!)

In [32]:
import os
import sys
import time
import numpy as np
import scipy.ndimage as nd
import matplotlib.pyplot as plt

%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


---
# Reading images into python

There are many ways to read images into a usable data structure in python.  Lots of good modules exist, some of the most popular being:<br>
. scipy.ndimage<br>
. PIL (python imaging library)<br>
. scikit-image<br>
. opencv<br>
<br>
Each of these has its pros and cons, we'll almost exclusively be using scipy.ndimage since we can load the data directly into a numpy array and it lets us stay "close" to the data.<br>
<br>
This notebook is running in a directory with a subdirectory called "images" where the Mona Lisa image is stored.  To load it into a numpy array,

In [23]:
dpath = 'images'
fname = 'ml.jpg'
infile = os.path.join(dpath,fname)
img_ml = nd.imread(infile)

We now have the image ready to use as a numpy array (with all of its associated methods).  Some important things to note about this array though are its shape and type,

In [24]:
print("shape : {0}\ntype  : {1}".format(img_ml.shape,img_ml.dtype))

shape : (954, 640, 3)
type  : uint8


---

# 3D arrays, data types, and heatmaps

While grayscale images are 2D arrays, color images are 3D (... and sometimes 4D) data <i>cubes</i>.  Typically, these are stored as 8-bit unsigned integers meaning that the value a given pixel can take is between $0$ and $255 = 2^8 - 1$.<br>
<br>
Fortunately, when matplotlib sees a 3D data cube it knows what to do:

In [25]:
ysize = 10.
xsize = ysize*float(img_ml.shape[1])/float(img_ml.shape[0])

fig5, ax5 = plt.subplots(num=5,figsize=[xsize,ysize])
fig5.subplots_adjust(0,0,1,1)
ax5.axis('off')
im5 = ax5.imshow(img_ml)
fig5.canvas.draw()

CAUTION: matplotlib always "interprets" 3D data cubes as being unsigned 8-bit integers.  If your 3D data cube is <i>not</i> an unsigned 8-bit integer, the color matching will fail,

In [26]:
im5.set_data(1.0*img_ml)
fig5.canvas.draw()

Let's say, for example, that you wanted to make the whole image fainter by a factor of $4$.  You can multiply by $0.25$ but be careful to change the type back to a uint8:

In [27]:
fig6, ax6 = plt.subplots(1,3,num=6,figsize=[3*xsize,ysize])
fig6.subplots_adjust(0,0,1,1,0,0)
[i.axis('off') for i in ax6]
im6a = ax6[0].imshow(img_ml)
im6b = ax6[1].imshow(0.25*img_ml)
im6c = ax6[2].imshow((0.25*img_ml).astype(np.uint8))
fig6.canvas.draw()

Whenever processing an image, always use the highest precision you can and only go back to 8-bit integers for visualizations.<br>
<br>
Notice though what matplotlib does when the input array is <i>not</i> a 3D data cube:

In [28]:
img_ml_L = img_ml.mean(2) # convert to gray scale by taking the mean across the color axis
print("img_ml_L\n  shape : {0}\n  type  : {1}".format(img_ml_L.shape,img_ml_L.dtype))

fig7, ax7 = plt.subplots(num=7,figsize=[xsize,ysize])
fig7.subplots_adjust(0,0,1,1)
ax7.axis('off')
im7 = ax7.imshow(img_ml_L)
fig7.canvas.draw()

img_ml_L
  shape : (954, 640)
  type  : float64


Here the data is a 2D 64-bit float.  In this case matplotlib creates a heatmap and intensity is preserved regardless of amplitude.

---

### Cropping, negative, and overplotting

1. Find a jpg from the internet and download it
2. Load it into python using scipy.ndimage
3. Display the full image
4. Display only the upper left corner
5. Display only the lower right corner
6. Display only the central half of the image
7. Diplay a negative of the full image
8. Reset the right half of the image as the negative of itself
9. Plot a step function with a transition at ncol/2 and height nrow
10. Overshow the result of step 8

---

## Advanced (and relational) indexing: Part 1

Numpy array indexing is fast and allows you to manipulate images with only a few lines of code.  As an example, we'll create the "spotlight" widget.

First, it is often useful to create row and column index "maps":

In [40]:
dpath = 'images'
fname = 'ml.jpg'
infile = os.path.join(dpath,fname)
img_ml = nd.imread(infile)

nrow, ncol = img_ml.shape[:2]
rind = np.arange(nrow*ncol).reshape(nrow,ncol) // ncol
cind = np.arange(nrow*ncol).reshape(nrow,ncol) % ncol

In [41]:
fig8, ax8 = plt.subplots(1,2)
ax8[0].imshow(rind)
ax8[0].grid(0)
ax8[1].imshow(cind)
ax8[1].grid(0)

which will let us index on positions in the image through "relational" indexing.  For example, we can create a mask for our image,

In [42]:
mask = np.dstack([(rind<500).astype(np.uint8) for i 
                  in range(3)])

In [43]:
mask.shape

(954, 640, 3)

In [44]:
ysize = 10.
xsize = ysize*float(img_ml.shape[1]) / \
    float(img_ml.shape[0])

fig9, ax9 = plt.subplots(num=9,
                         figsize=[xsize,ysize])
fig9.subplots_adjust(0,0,1,1)
ax9.axis('off')
im9 = ax9.imshow(img_ml*mask)
fig9.canvas.draw()

So, let's make a circular aperature around a point in the image:

In [45]:
rm, cm = 244, 302
dist = np.sqrt((rind-rm)**2+(cind-cm)**2)

im9.set_data(dist)
im9.set_clim(0,500)
fig9.canvas.draw()

In [46]:
dist.shape

(954, 640)

In [47]:
mask = np.zeros(img_ml.shape,dtype=np.uint8)
mask[dist<=100] = [1,1,1]

im9.set_data(255*mask)
fig9.canvas.draw()

In [48]:
im9.set_data(img_ml*mask)
fig9.canvas.draw()

#### using Matplotlib's ginput() function

The ginput() function let's you click on a point and grab its location from the matplotlib window.  For example,

In [49]:
xpos, ypos = fig9.ginput()[0]

print("xpos = {0}\nypos = {1}".format(xpos,ypos))

xpos = 301.589552239
ypos = 244.858208955


In [50]:
print fig9.ginput(3)

[(349.35074626865668, 181.57462686567169), (290.8432835820895, 324.85820895522386), (290.8432835820895, 324.85820895522386)]


In [51]:
cpos, rpos = [int(round(i)) for i in 
              fig9.ginput()[0]]

print("rpos = {0}\ncpos = {1}".format(rpos,cpos))

rpos = 197
cpos = 287


---

## Advanced (and relational) indexing: Part 2

In addition to indexing based on position within an image, we can index on pixel <i>values</i> which is useful for selecting subcomponents of an image that are not conected compoenents.  As an example, let's create an image consisting of a blue circle and line:

In [52]:
rind = np.arange(20000).reshape(200,100) // 100
cind = np.arange(20000).reshape(200,100) % 100
two_ln = ((rind>130) & (rind<150)) | \
    (np.sqrt((rind-50)**2+(cind-50)**2)<20)
two_ln_c = np.dstack([64*two_ln, 128*two_ln, 192*two_ln]
                     ).astype(np.uint8)

Plotting it:

In [53]:
fig0, ax0 = plt.subplots(num=0,figsize=[5,10])
fig0.subplots_adjust(0,0,1,1)
ax0.grid('off')
im0 = ax0.imshow(two_ln_c)

Now identify the pixels in 2D which have the specified color and set those pixels to brown,

In [54]:
index = (two_ln_c[:,:,0]==64) & \
    (two_ln_c[:,:,1]==128) & \
    (two_ln_c[:,:,2]==192)
two_ln_c[index] = [192,128,64]

and replot.

In [55]:
im0.set_data(two_ln_c)
fig0.canvas.draw()

In [56]:
plt.close(0)

---

## Example: Invasive Species

Freshkills is a 2,200 acre former landfill on Staten Island that is in the process of being converted into a park (Freshkills park).  There is significant contamination by an invasive species called phragmites.

#### CUSP PhD student Nick Johnson flew a balloon over Freshkills Park to image the extent of the phragmites invasion.

In [57]:
dpath = 'images'
dfile = 'phrag-annotated.png'
infile = os.path.join(dpath,dfile)

img = nd.imread(infile)[:,:,:3] # ignore the alpha channel

nrow, ncol = img.shape[:2]

In [58]:
xsize = 7.
ysize = xsize*float(nrow)/float(ncol)

fig0, ax0 = plt.subplots(num=0,figsize=[xsize,ysize])
fig0.subplots_adjust(0,0,1,1); ax0.axis('off')
im0 = ax0.imshow(img)
fig0.canvas.draw()

What separates the phragmites from the healthy vegetation is the <i>color</i>.  But we already know how to index on color, so let's get an idea of what color we're looking for.  If we zoom in on a region dominated by phragmites:

In [59]:
prow = [1460,1605]
pcol = [4015,4354]

ax0.add_patch(plt.Rectangle((pcol[0],prow[0]),
                           pcol[1]-pcol[0],prow[1]-prow[0],
              facecolor='none',edgecolor='orange',lw=2))
ax0.set_xlim(pcol[0]-100,pcol[1]+100)
ax0.set_ylim(prow[0]-100,prow[1]+100)
fig0.canvas.draw()

We can see that it's a bit bluer than a region with healthy vegetation:

In [60]:
hrow = [2650,2905]
hcol = [570,915]

ax0.add_patch(plt.Rectangle((hcol[0],hrow[0]),
                           hcol[1]-hcol[0],hrow[1]-hrow[0],
              facecolor='none',edgecolor='orange',lw=2))
ax0.set_xlim(hcol[0]-100,hcol[1]+100)
ax0.set_ylim(hrow[0]-100,hrow[1]+100)
fig0.canvas.draw()

In [75]:
ax0.set_xlim(0,ncol); ax0.set_ylim(nrow,0); fig0.canvas.draw()

To start understanding what kind of indexing we need to do, let's make some histograms.

In [61]:
stamp_ph = img[prow[0]:prow[1],pcol[0]:pcol[1]]
stamp_he = img[hrow[0]:hrow[1],hcol[0]:hcol[1]]

In [71]:
fig1, ax1 = plt.subplots(2,2,num=1,figsize=[2*xsize,xsize])
clrs = ['firebrick','olivedrab','royalblue']

[ax1[0,0].hist(stamp_ph[:,:,i].flatten(),bins=128,range=[0,255], 
               normed=True,facecolor=clrs[i]) for i in range(3)]
[ax1[1,0].hist(stamp_he[:,:,i].flatten(),bins=128,range=[0,255], 
               normed=True,facecolor=clrs[i]) for i in range(3)]
[i.set_yticklabels('') for j in ax1 for i in j]

ax1[0,0].set_ylabel('phragmites')
ax1[1,0].set_ylabel('healthy')
fig1.canvas.draw()

So the first thing we can do is cut on the ratio of green to blue since the healthy vegetation has a ratio >1.5 for (more or less) all pixels in our patch.

In [72]:
hind = (1.0*img[:,:,1])/(1.0*img[:,:,2]) < 1.8

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: RuntimeWarning: divide by zero encountered in divide
  if __name__ == '__main__':
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: RuntimeWarning: invalid value encountered in divide
  if __name__ == '__main__':
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: RuntimeWarning: invalid value encountered in less
  if __name__ == '__main__':


In [73]:
hind = (1.0*img[:,:,1])/(1.0*img[:,:,2] + (img[:,:,2]==0)) < 1.5

In [74]:
plt.close(2)
fig2, ax2 = plt.subplots(num=2,figsize=[xsize,ysize])
fig2.subplots_adjust(0,0,1,1); ax2.axis('off')
im2 = ax2.imshow(hind)
fig2.canvas.draw()

Looks like we're getting phragmites, dirt, and road.  Let's look at the histograms of the dirt and road:

In [76]:
drow = [2450,2880]
dcol = [1905,2235]
rrow = [1665,1790]
rcol = [3600,3725]

stamp_di = img[drow[0]:drow[1],dcol[0]:dcol[1]]
stamp_rd = img[rrow[0]:rrow[1],rcol[0]:rcol[1]]

for reg in [[drow,dcol],[rrow,rcol]]:
    ax0.add_patch(plt.Rectangle((reg[1][0],reg[0][0]),
                               reg[1][1]-reg[1][0],reg[0][1]-reg[0][0],
                  facecolor='none',edgecolor='orange',lw=2))
fig0.canvas.draw()

In [77]:
[ax1[0,1].hist(stamp_di[:,:,i].flatten(),bins=128,range=[0,255], 
               normed=True,facecolor=clrs[i]) for i in range(3)]
[ax1[1,1].hist(stamp_rd[:,:,i].flatten(),bins=128,range=[0,255], 
               normed=True,facecolor=clrs[i]) for i in range(3)]

ax1[0,1].set_ylabel('dirt')
ax1[1,1].set_ylabel('road')

fig1.canvas.draw()

It looks like "dirt" has a high red-to-blue ratio compared to phragmites, so we update our definition of phragmites colors:

In [78]:
hind = ((1.0*img[:,:,1])/(1.0*img[:,:,2] + (img[:,:,2]==0)) < 1.5) & \
    ((1.0*img[:,:,0])/(1.0*img[:,:,2] + (img[:,:,2]==0)) < 1.2)

In [79]:
im2.set_data(hind)
fig2.canvas.draw()

Lastly, the road mostly has values $>200$ while the phragmites are $<200$ (also let's use the fact that phragmites have red values which are $<150$ and the lines are black).

In [80]:
hind = ((1.0*img[:,:,1])/(1.0*img[:,:,2] + (img[:,:,2]==0)) < 1.5) & \
    ((1.0*img[:,:,0])/(1.0*img[:,:,2] + (img[:,:,2]==0)) < 1.2) & \
    (img.max(2)<200) & \
    (img[:,:,0]<150) & \
    (img.max(2)>50)

In [81]:
im2.set_data(hind)
fig2.canvas.draw()

---

## Multiple frames (videos)

Unlike single still images, multiple images of the same scene over time let you measure **time dependent** features (e.g., motion, color changes, etc.).  We will deal with already extracted frames from a video here, but an example about how that extraction can be done in python with OpenCV is <a href="http://tobilehman.com/blog/2013/01/20/extract-array-of-frames-from-mp4-using-python-opencv-bindings/">here</a>. 

### DOT camera analysis

The Department of Transportation has hundreds of traffic cameras located around the city.  Using this script:

    import os
    import time

    nframe   = 100
    cmd_wget = "wget http://207.251.86.238/cctv391.jpg"
    cmd_mv   = "mv cctv391.jpg images/dot/cctv391_{0:03}.jpg"

    for ii in range(nframe):
        os.system(cmd_wget)
        os.system(cmd_mv.format(ii))
        time.sleep(1)

I grabbed 100 frames from camera overlooking the Manhattan bridge (and Lower East Side):

In [82]:
path1 = 'images'
path2 = 'dot'
dpath = os.path.join(path1,path2)
flist = [os.path.join(dpath,i) for i in 
         sorted(os.listdir(os.path.join(dpath)))]

for i in flist:
    print(i)

images/dot/cctv391_000.jpg
images/dot/cctv391_001.jpg
images/dot/cctv391_002.jpg
images/dot/cctv391_003.jpg
images/dot/cctv391_004.jpg
images/dot/cctv391_005.jpg
images/dot/cctv391_006.jpg
images/dot/cctv391_007.jpg
images/dot/cctv391_008.jpg
images/dot/cctv391_009.jpg
images/dot/cctv391_010.jpg
images/dot/cctv391_011.jpg
images/dot/cctv391_012.jpg
images/dot/cctv391_013.jpg
images/dot/cctv391_014.jpg
images/dot/cctv391_015.jpg
images/dot/cctv391_016.jpg
images/dot/cctv391_017.jpg
images/dot/cctv391_018.jpg
images/dot/cctv391_019.jpg
images/dot/cctv391_020.jpg
images/dot/cctv391_021.jpg
images/dot/cctv391_022.jpg
images/dot/cctv391_023.jpg
images/dot/cctv391_024.jpg
images/dot/cctv391_025.jpg
images/dot/cctv391_026.jpg
images/dot/cctv391_027.jpg
images/dot/cctv391_028.jpg
images/dot/cctv391_029.jpg
images/dot/cctv391_030.jpg
images/dot/cctv391_031.jpg
images/dot/cctv391_032.jpg
images/dot/cctv391_033.jpg
images/dot/cctv391_034.jpg
images/dot/cctv391_035.jpg
images/dot/cctv391_036.jpg
i

In [83]:
imgs = np.array([nd.imread(i) for i in flist])
nimg, nrow, ncol = imgs.shape[0:3]
xs = 6.5
ys = 2*xs*float(nrow)/float(ncol)

plt.close(0)
fig0, ax0 = plt.subplots(2,1,num=0,figsize=[xs,ys])
fig0.subplots_adjust(0,0,1,1,0,0)
[i.axis('off') for i in ax0]
im0a = ax0[0].imshow(imgs[0])
fig0.canvas.draw()

In [84]:
for img in imgs:
    im0a.set_data(img)
    fig0.canvas.draw()

We'd like to **roughly** count the number of cars on this section of the highway.  Let's first look at frame differences:

In [85]:
im0b = ax0[1].imshow(imgs[1].mean(-1) - imgs[0].mean(-1))
fig0.canvas.draw()

In [86]:
for ii in range(1,nimg):
    im0a.set_data(imgs[ii])
    im0b.set_data(imgs[ii].mean(-1) - imgs[ii-1].mean(-1))
    fig0.canvas.draw()

Or taking the absolute value (note: if we were to count off of this we'd be roughly double counting...)

In [87]:
im0b.set_clim(0,128)

for ii in range(1,nimg):
    im0a.set_data(imgs[ii])
    im0b.set_data(np.abs(imgs[ii].mean(-1) - imgs[ii-1].mean(-1)))
    fig0.canvas.draw()

Let's do some thresholding:

In [88]:
dimg = np.zeros([nrow,ncol])
im0b.set_clim(0,1)

for ii in range(1,nimg):
    dimg[:,:] = np.abs(imgs[ii].mean(-1) - imgs[ii-1].mean(-1))
    im0a.set_data(imgs[ii])
    im0b.set_data(dimg>40)
    fig0.canvas.draw()

### Background subtraction

But again, we're double counting and adding noise.  Another method is to subtract the "mean image" from each frame:

In [89]:
mimg = imgs.mean(0)

im0a.set_data(mimg.clip(0,255).astype(np.uint8))
fig0.canvas.draw()

In [90]:
for ii in range(1,nimg):
    im0a.set_data(imgs[ii])
    im0b.set_data(np.abs(1.0*imgs[ii] - mimg).clip(0,255).astype(np.uint8))
    fig0.canvas.draw()

Let's take the max of this difference in color space:

In [91]:
im0b.set_clim([0,255])

for ii in range(1,nimg):
    im0a.set_data(imgs[ii])
    im0b.set_data(np.abs(1.0*imgs[ii] - mimg).max(-1))
    fig0.canvas.draw()

Again, we can do some clipping, but let's take a look at the distribution of brightnesses

In [105]:
plt.close(1)

fig1,ax1 = plt.subplots(num=1)
ax1.hist(np.log10(np.abs(1.0*imgs - mimg).max(-1).flatten()+1.0), bins=255,
        facecolor="darkred",lw=0)
ax1.set_xlabel("brightness")
ax1.set_ylabel("PDF")
ax1.grid(1)
fig1.canvas.draw()

The logarithm of the difference values for the objects is something like $>1.5 \approx 31$:

In [94]:
thr = 30
im0b.set_clim([0,1])

for ii in range(1,nimg):
    im0a.set_data(imgs[ii])
    im0b.set_data(np.abs(1.0*imgs[ii] - mimg).max(-1)>thr)
    fig0.canvas.draw()

In [95]:
bdilation = nd.morphology.binary_dilation
berosion = nd.morphology.binary_erosion

In [96]:
thr = 30
im0b.set_clim([0,1])
fgr = np.zeros([nrow,ncol],dtype=int)

for ii in range(1,nimg):
    fgr[:,:] = bdilation(berosion(np.abs(1.0*imgs[ii] - mimg).max(-1)>thr),iterations=2)
    im0a.set_data(imgs[ii])
    im0b.set_data(fgr)
    fig0.canvas.draw()

We can use our trick of a row map to generate a mask,

In [97]:
col_mask = (np.arange(nrow*ncol).reshape(nrow,ncol)%ncol)<250

In [98]:
thr = 30
im0b.set_clim([0,1])
fgr = np.zeros([nrow,ncol],dtype=int)

for ii in range(1,nimg):
    fgr[:,:] = bdilation(berosion(np.abs(1.0*imgs[ii] - mimg).max(-1)>thr),iterations=2)
    im0a.set_data(imgs[ii])
    im0b.set_data(fgr*col_mask)
    fig0.canvas.draw()

Lastly, we count the number of detected objects:

In [99]:
count = ax0[1].text(0,0,"# of cars: ",va='top',
                    fontsize=20,color='white')
fig0.canvas.draw()

In [100]:
meas_label = nd.measurements.label

In [101]:
thr = 30
sz_thr = 20 # only count objects greater than a size threshold
im0b.set_clim([0,1])
fgr = np.zeros([nrow,ncol],dtype=int)

for ii in range(1,nimg):
    fgr[:,:] = bdilation(berosion(np.abs(1.0*imgs[ii] - mimg).max(-1)>thr),iterations=2)
    labs = meas_label(fgr*col_mask)
    ncar = sum([1*((labs[0]==lab).sum()>sz_thr) for lab in range(1,labs[1]+1)])

    im0a.set_data(imgs[ii])
    im0b.set_data(fgr*col_mask)
    count.set_text("# of cars: {0}".format(ncar))
    fig0.canvas.draw()

---

## Closing thoughts and recap

Below are some of the key concepts we've worked through.  They are broadly applicable, even well beyond the field of image processing and computer vision:

- best practices for filesystem handling and namespaces
- vectorized coding (indexing arrays to eliminate for loops)
- object oriented plotting with matplotlib (for interactive plots)
- thresholding for data subselection and masking
- derivatives and edge detection through array shifts
- combining 2D data with time series information